In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms.v2 as v2
from torchvision.models import GoogLeNet_Weights

from data_prep import DermNet, get_dataloaders
from googlenet_scalp import GoogLeNet_Scalp
from model_trainer import train_model

# hyperparameters
batch_size = 64
lr = 1e-3
eps = 1e-4
weight_decay = 1e-3
step_size = 7
gamma = 0.1
num_epochs = 25

# optimizations
num_workers = 12
pin_memory = True
torch.backends.cudnn.benchmark = True
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# get transform and augment
transform = GoogLeNet_Weights.DEFAULT.transforms()
augmenter = v2.AugMix()

# get dataset and data loaders
dataset = DermNet(transform=transform)
num_classes = len(dataset.classes)
train_loader, val_loader = get_dataloaders(
    dataset=dataset, transform=transform, batch_size=batch_size,
    num_workers=num_workers, pin_memory=pin_memory
)
dataloaders = {'train': train_loader, 'val': val_loader}

# setup model
model_ft = GoogLeNet_Scalp(device=device, num_classes=num_classes)

criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.Adam(model_ft.parameters(), lr=lr, eps=eps, weight_decay=weight_decay)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer_ft, step_size=step_size, gamma=gamma)

model_ft = train_model(model_ft, dataloaders, criterion, optimizer_ft, lr_scheduler,
                        num_epochs=num_epochs, device=device, augmenter=augmenter)
# save model
torch.save(model_ft.state_dict(), 'weights/model_checkpoint.pt')

Epoch 1/25 | Learning Rate 0.001
----------


/home/jo-wubuntu/.cache/pypoetry/virtualenvs/hair-deisease-cnn-project-sPLV6DJP-py3.10/lib/python3.10/site-packages/torch/nn/modules/conv.py:456: UserWarning: Applied workaround for CuDNN issue, install nvrtc.so (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:80.)
  return F.conv2d(input, weight, bias, self.stride,


train Loss: 0.9117 Acc: 0.5337
val Loss: 0.8187 Acc: 0.6667

Epoch 2/25 | Learning Rate 0.001
----------
train Loss: 0.4139 Acc: 0.8427
val Loss: 0.8076 Acc: 0.6889

Epoch 3/25 | Learning Rate 0.001
----------
train Loss: 0.1779 Acc: 0.9438
val Loss: 1.0335 Acc: 0.6667

Epoch 4/25 | Learning Rate 0.001
----------
train Loss: 0.1834 Acc: 0.9326
val Loss: 0.7443 Acc: 0.8222

Epoch 5/25 | Learning Rate 0.001
----------
train Loss: 0.1037 Acc: 0.9719
val Loss: 1.1559 Acc: 0.7333

Epoch 6/25 | Learning Rate 0.001
----------
train Loss: 0.0633 Acc: 0.9888
val Loss: 1.6511 Acc: 0.6667

Epoch 7/25 | Learning Rate 0.001
----------
train Loss: 0.0891 Acc: 0.9607
val Loss: 0.5927 Acc: 0.8222

Epoch 8/25 | Learning Rate 0.0001
----------
train Loss: 0.0582 Acc: 0.9831
val Loss: 0.8701 Acc: 0.7333

Epoch 9/25 | Learning Rate 0.0001
----------
train Loss: 0.0457 Acc: 0.9888
val Loss: 0.5889 Acc: 0.8222

Epoch 10/25 | Learning Rate 0.0001
----------
train Loss: 0.0607 Acc: 0.9719
val Loss: 0.5485 Acc